<a href="https://colab.research.google.com/github/Gutobastos/imersao-iA-3-Alura/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai
%pip -q install google-adk

In [6]:
import os

# E necessário inserir a chave na variável do seu ambiente com sua GOOGLE_API_KEY
google_api_key = os.environ.get('GOOGLE_API_KEY')
if google_api_key:
    print("Chave da API do Google encontrada nas variáveis de ambiente do seu PC.\n")
else:
    print("Chave da API do Google não encontrada nas variáveis de ambiente e tentaremos pelo GOOGLE COLAB.\n")
    #from google.colab import userdata
    #os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Chave da API do Google encontrada nas variáveis de ambiente do seu PC.



In [7]:
from google import genai
client = genai.Client()
modelo = "gemini-2.0-flash"

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [10]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [11]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: BUSCADOR --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações no Google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pesquisa de móveis e deve usar a ferramenta de 
      busca do Google (google_search), procure o móvel específico e que seja o mais barato, 
      tenha garantia e boa referência de reputação e avaliação a partir da data de hoje.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  moveis = call_agent(buscador, entrada_do_agente_buscador)
  return moveis

In [ ]:
################################################
# --- Agente 2: ORÇAMENTO --- #
################################################
def agente_cotador(topico, moveis_buscados):
    planejador = Agent(
        name="agente_cotacao",
        model="gemini-2.0-flash",
        description="Agente cotação de preços",
        tools=[google_search],
        instruction="""
        Você é um filtro de pesquisa, especialista em encontrar o móvel com melhor custo 
        benefícioa partir da data de hoje. Você deve usar a feramenta de busca do Google (google_search) e dar 
        prioridade preço mais baixo.
        Escolha de 1 até 5 móveis.
        """
    )
    entrada_do_agente_cotador = f"Tópico:{topico}\nLançamentos buscados: {moveis_buscados}"

    moveis_do_post = call_agent(planejador, entrada_do_agente_cotador)
    return moveis_do_post

In [ ]:
######################################
# --- Agente 3: AVALIADOR --- #
######################################
def agente_avaliador(topico, avaliacao_de_post):
    redator = Agent(
        name="agente_avaliador",
        model="gemini-2.5-pro-preview-03-25",
        description="Agente avaliador de posts",
        instruction="""
            Você é um avaliador de controle de qualidade, você deve priorizar 
            as médias de avaliações, você deve usar a feramenta de busca do Google (google_search).
            Escolha até 5 produtos que tenha a melhor média de avaliação e 3 melhores 
            comentários de clientes se tiver.
            """
    )
    entrada_do_agente_avaliador = f"Tópico: {topico}\nPlano de post: {avaliacao_de_post}"

    rascunho = call_agent(redator, entrada_do_agente_avaliador)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: RESULTADO --- #
##########################################
def agente_retorno(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        description="Agente revisor de post para redes sociais.",
        instruction="""
            Você é um vendedor e dará informação da mais completa sobre o produto, 
            você é especializado vendas varejista, com foco no melhor móvel, com melhor custo, benefício e com menor prazo de entrega se tiver.
            Por ter um público, entre 20 e 50 anos, use um tom de escrita adequado, respeitoso e confiante.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho avaliado, se estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """
    )
    entrada_do_agente_retorno = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"

    texto_final = call_agent(revisor, entrada_do_agente_retorno)
    return texto_final

In [ ]:
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))